# <center>ZUM</center>
# <center> Projekt -- dokumentacja końcowa </center>

## <center> Jan Budziński -- 310609 </center>
## <center> Jarosław Nachyła -- nr_indeksu </center>

## 2. Implementacja

### 2.1. Nienadzorowana detekcja anomalii

W ramach projektu zaimplementowano klasę <i>AnomalyDetector</i>, będącą opakowaniem na algorytmy grupowania KMeans, DBSCAN oraz AgglomerativeClustering, jak również na miary niepodobieństwa euklidesową, Mahalanobisa i Manhattan. Celem tejże klasy jest stworzenie łatwego w obsłudze środowiska testowego, które w miarę możliwości (niektóre algorytmy różnią się wejściami) będzie jednorodne w użyciu dla każdego z porównywanych algorytmów i miar.
Klasa ta zawiera następujące metody:

- fit -- dopasowuje model do danych wejściowych,
- fit_predict -- wykonuje trening modelu i jednocześnie przewidująca klasy i określająca dystanse od centrów klastrów dla danych wejściowych,
- transform_distances -- określa, czy dane wejściowe są anomaliami na podstawie dystansu otrzymanego z wybranej miary niepodobieństwa,
- transform_labels -- dla liczby klastrów większych niż 2, metoda ta zmienia przypisanie do wszystkich klastrów poza najliczniejszym w anomalię.


Dzięki tym metodom użytkownik jest w stanie w łatwy sposób testować różne modele i miary niepodobieństwa, zmieniając wyłącznie jeden parametr w kodzie.

Kod implementujący tę klasę jest w pliku anomaly_detector.py.

Ponadto utworzono klasę <i>AnomalyDetectorEvaluator</i> zawierającą metody obliczające metryki potrzebne do ewaluacji wytrenowanych modeli.

Testowane metryki to:

- dokładność (accuracy)
- precyzja (precision)
- czułość (recall)
- pole pod wykresem PRC

Wszystkie te metryki zostały dokładniej opisane w dokumentacji wstępnej. Jednakże, z uwagi na fakt, iż dane z założenia są wysoce niezbalansowane (jako że są to dane anomalii, to wejścia o pozytywnej klasie stanowią poniżej 1% wszystkich) uznano, że zwykła dokładność może niewiele powiedzieć, jako że przypisanie wszystkim danym klasy negatywnej pozwala osiągnąć powyżej 99% poprawnych predykcji. W tym celu dodano metrykę dokładności wykrywania outlierów, która testuje, ile spośród prawdziwych outlierów zostało poprawnie zidentyfikowanych. 

Kod z klasą AnomalyDetectorEvaluator znajduje się w pliku metrics.py.

### 2.2. Jednoklasowy klasyfikator

## 3. Eksperymenty

### 3.1 Porównanie modeli grupowania

W ramach pierwszego eksperymentu wykonano testy porównawcze iloczynu kartezjańskiego 3 wybranych modeli, 3 miar niepodobieństwa i 2 zbiorów danych, czyli łącznie 18 pojedynczych testów.

Podczas przeprowadzania eksperymentu wystąpiły pewne problemy. Mianowicie, algorytm AgglomerativeClustering cechuje się wysokimi złożonościami: złożoność obliczeniowa to $O(n^2)$, a czasowa to $O(n^3)$. Z tego względu trening na całych zbiorach danych, a w szczególności na zbiorze HTTP był niemożliwy. W związku z tym zastosowano subsampling.

Wyniki testów przedstawione są w poniższych tabelach.

**_Zbiór Shuttle_**

| miara \ model | **KMeans** | **DBSCAN** | **AgglomerativeClustering** |
| ------------- | ---------- | ---------- | --------------------------- |
| euklidesowa   |            |            |                             |
| Mahalanobisa  |            |            |                             |
| Manhattan     |            |            |                             |

**_Zbiór HTTP_**

| miara \ model | **KMeans** | **DBSCAN** | **AgglomerativeClustering** |
| ------------- | ---------- | ---------- | --------------------------- |
| euklidesowa   |            |            |                             |
| Mahalanobisa  |            |            |                             |
| Manhattan     |            |            |                             |

### 3.2. Grupowanie vs klasyfikacja jednoklasowa

### 3.3. Grupowanie z MetaCost

In [1]:
import pandas as pd
import numpy as np
import scipy.io
from anomaly_detector import AnomalyDetector
from metrics import AnomalyDetectorEvaluator

In [2]:
shuttle = scipy.io.loadmat("shuttle.mat")
shuttle_data = pd.DataFrame(shuttle['X'])
shuttle_eval = pd.DataFrame(shuttle['y'])
http_data = pd.read_csv("http_train.csv")
http_eval = pd.read_csv("http_eval.csv")

In [3]:
shuttle_eval = np.array(shuttle_eval).flatten()
http_eval = np.array(http_eval).flatten()

In [27]:
#shuttle_data

In [2]:
import racplusplus

ModuleNotFoundError: No module named 'racplusplus'

In [5]:
http_data = http_data.to_numpy()

In [1]:
labels = racplusplus.rac(http_data, 0.95, None, 10000, 8, "cosine")


# Predict clusters
labels_http_racpp = racpp.labels_

# Compute distances or anomaly scores if available
distances_http_racpp = racpp.distances_  # Hypothetical attribute

# Transform labels and distances for anomaly detection
transformed_labels = AnomalyDetector.transform_labels(labels_http_racpp)
transformed_distances = AnomalyDetector.transform_distances(distances_http_racpp)

# Evaluate the model
evaluator_http_racpp = AnomalyDetectorEvaluator(http_eval, transformed_labels, transformed_distances)
accuracy_http_racpp = evaluator_http_racpp.calculate_accuracy()
recall_http_racpp = evaluator_http_racpp.calculate_recall()
precision_http_racpp = evaluator_http_racpp.calculate_precision()
auc_pr_http_racpp = evaluator_http_racpp.calculate_auc_pr()

# Print the results
print(accuracy_http_racpp, recall_http_racpp, precision_http_racpp, auc_pr_http_racpp)


NameError: name 'racplusplus' is not defined

In [ ]:
kmeans = AnomalyDetector(model="kmeans", n_clusters=2)
labels_shuttle_kmeans, distances_shuttle_kmeans = kmeans.fit_predict(data=shuttle_data)
labels_shuttle_kmeans = AnomalyDetector.transform_labels(labels_shuttle_kmeans)
distances_shuttle_kmeans = AnomalyDetector.transform_distances(distances_shuttle_kmeans)
evaluator_shuttle_kmeans = AnomalyDetectorEvaluator(shuttle_eval, labels_shuttle_kmeans, distances_shuttle_kmeans)
accuracy_shuttle_kmeans = evaluator_shuttle_kmeans.calculate_accuracy()
recall_shuttle_kmeans = evaluator_shuttle_kmeans.calculate_recall()
precision_shuttle_kmeans = evaluator_shuttle_kmeans.calculate_precision()
auc_pr_shuttle_kmeans = evaluator_shuttle_kmeans.calculate_auc_pr()
accuracy_shuttle_kmeans, recall_shuttle_kmeans, precision_shuttle_kmeans, auc_pr_shuttle_kmeans

In [5]:
evaluator_shuttle_kmeans.imbalanced_metrics()

(                         Positive Prediction        Negative Prediction
 0  Positive Class   True Positive (TP) 4.94%  False Negative (FN) 2.22%
 1  Negative Class  False Positive (FP) 0.29%  True Negative (TN) 92.56%,
                Metric   Value
 0     Positive Recall  69.01%
 1     Negative Recall  99.68%
 2  Positive Precision  94.39%
 3  Negative Precision  97.66%)

In [ ]:
evaluator_shuttle_kmeans.calculate_outliers_accuracy()

In [ ]:
dbscan = AnomalyDetector(model="dbscan", )
labels_shuttle_dbscan, distances_shuttle_dbscan = dbscan.fit_predict(data=shuttle_data)
labels_shuttle_dbscan = AnomalyDetector.transform_labels(labels_shuttle_dbscan)
distances_shuttle_dbscan = AnomalyDetector.transform_distances(distances_shuttle_dbscan)
evaluator_shuttle_dbscan = AnomalyDetectorEvaluator(shuttle_eval, labels_shuttle_dbscan, distances_shuttle_dbscan)
accuracy_shuttle_dbscan = evaluator_shuttle_dbscan.calculate_accuracy()
recall_shuttle_dbscan = evaluator_shuttle_dbscan.calculate_recall()
precision_shuttle_dbscan = evaluator_shuttle_dbscan.calculate_precision()
auc_pr_shuttle_dbscan = evaluator_shuttle_dbscan.calculate_auc_pr()
accuracy_shuttle_dbscan, recall_shuttle_dbscan, precision_shuttle_dbscan, auc_pr_shuttle_dbscan

In [ ]:
evaluator_shuttle_dbscan.calculate_outliers_accuracy()

In [ ]:

batch_size = 10000

all_labels = []
all_distances = []

for start in range(0, len(shuttle_data), batch_size):
    end = start + batch_size
    batch_data = shuttle_data[start:end]

    agglomerative = AnomalyDetector(model="agglomerative", n_clusters=2)
    labels_batch, distances_batch = agglomerative.fit_predict(data=batch_data)

    all_labels.extend(AnomalyDetector.transform_labels(labels_batch))
    all_distances.extend(AnomalyDetector.transform_distances(distances_batch))

all_labels = np.array(all_labels)
all_distances = np.array(all_distances)

evaluator_shuttle_agglomerative = AnomalyDetectorEvaluator(shuttle_eval, all_labels, all_distances)
accuracy_shuttle_agglomerative = evaluator_shuttle_agglomerative.calculate_accuracy()
recall_shuttle_agglomerative = evaluator_shuttle_agglomerative.calculate_recall()
precision_shuttle_agglomerative = evaluator_shuttle_agglomerative.calculate_precision()
auc_pr_shuttle_agglomerative = evaluator_shuttle_agglomerative.calculate_auc_pr()

accuracy_shuttle_agglomerative, recall_shuttle_agglomerative, precision_shuttle_agglomerative, auc_pr_shuttle_agglomerative

In [ ]:
evaluator_shuttle_agglomerative.calculate_outliers_accuracy()

In [ ]:
agglomerative = AnomalyDetector(model="agglomerative", n_clusters=2)
labels_shuttle_agglomerative, distances_shuttle_agglomerative = agglomerative.fit_predict(data=shuttle_data)
labels_shuttle_agglomerative = AnomalyDetector.transform_labels(labels_shuttle_agglomerative)
distances_shuttle_agglomerative = AnomalyDetector.transform_distances(distances_shuttle_agglomerative)
evaluator_shuttle_agglomerative = AnomalyDetectorEvaluator(shuttle_eval, labels_shuttle_agglomerative, distances_shuttle_agglomerative)
accuracy_shuttle_agglomerative = evaluator_shuttle_agglomerative.calculate_accuracy()
recall_shuttle_agglomerative = evaluator_shuttle_agglomerative.calculate_recall()
precision_shuttle_agglomerative = evaluator_shuttle_agglomerative.calculate_precision()
auc_pr_shuttle_agglomerative = evaluator_shuttle_agglomerative.calculate_auc_pr()
accuracy_shuttle_agglomerative, recall_shuttle_agglomerative, precision_shuttle_agglomerative, auc_pr_shuttle_agglomerative

In [ ]:
kmeans = AnomalyDetector(model="kmeans", n_clusters=2)
labels_http_kmeans, distances_http_kmeans = kmeans.fit_predict(data=http_data)
labels_http_kmeans = AnomalyDetector.transform_labels(labels_http_kmeans)
distances_http_kmeans = AnomalyDetector.transform_distances(distances_http_kmeans)
evaluator_http_kmeans = AnomalyDetectorEvaluator(http_eval, labels_http_kmeans, distances_http_kmeans)
accuracy_http_kmeans = evaluator_http_kmeans.calculate_accuracy()
recall_http_kmeans = evaluator_http_kmeans.calculate_recall()
precision_http_kmeans = evaluator_http_kmeans.calculate_precision()
auc_pr_http_kmeans = evaluator_http_kmeans.calculate_auc_pr()
accuracy_http_kmeans, recall_http_kmeans, precision_http_kmeans, auc_pr_http_kmeans

In [ ]:
evaluator_http_kmeans.calculate_outliers_accuracy()

In [ ]:
%load_ext memory_profiler

In [ ]:
dbscan = AnomalyDetector(model="dbscan", )
labels_http_dbscan, distances_http_dbscan = dbscan.fit_predict(data=http_data)
labels_http_kmeans = AnomalyDetector.transform_labels(labels_http_kmeans)
distances_http_kmeans = AnomalyDetector.transform_distances(distances_http_kmeans)
evaluator_http_dbscan = AnomalyDetectorEvaluator(http_eval, labels_http_dbscan, distances_http_dbscan)
accuracy_http_dbscan = evaluator_http_dbscan.calculate_accuracy()
recall_http_dbscan = evaluator_http_dbscan.calculate_recall()
precision_http_dbscan = evaluator_http_dbscan.calculate_precision()
auc_pr_http_dbscan = evaluator_http_dbscan.calculate_auc_pr()

In [ ]:
evaluator_http_dbscan.calculate_outliers_accuracy()

In [ ]:
agglomerative = AnomalyDetector(model="agglomerative", n_clusters=2)
labels_http_agglomerative, distances_http_agglomerative = agglomerative.fit_predict(data=http_data)
evaluator_http_agglomerative = AnomalyDetectorEvaluator(http_eval, labels_http_agglomerative, distances_http_agglomerative)
accuracy_http_agglomerative = evaluator_http_agglomerative.calculate_accuracy()
recall_http_agglomerative = evaluator_http_agglomerative.calculate_recall()
precision_http_agglomerative = evaluator_http_agglomerative.calculate_precision()
auc_pr_http_agglomerative = evaluator_http_agglomerative.calculate_auc_pr()

In [ ]:
evaluator_http_agglomerative.calculate_outliers_accuracy()

In [ ]:
batch_size = 20000


for start in range(0, len(http_data), batch_size):
    end = start + batch_size
    batch_data = http_data[start:end]

    agglomerative = AnomalyDetector(model="agglomerative", n_clusters=2)
    agglomerative.fit(data=batch_data)

all_labels, all_distances = agglomerative.predict(data=http_data)

evaluator_http_agglomerative = AnomalyDetectorEvaluator(http_eval, all_labels, all_distances)
accuracy_http_agglomerative = evaluator_http_agglomerative.calculate_accuracy()
recall_http_agglomerative = evaluator_http_agglomerative.calculate_recall()
precision_http_agglomerative = evaluator_http_agglomerative.calculate_precision()
auc_pr_http_agglomerative = evaluator_http_agglomerative.calculate_auc_pr()

accuracy_http_agglomerative, recall_http_agglomerative, precision_http_agglomerative, auc_pr_http_agglomerative

In [ ]:
evaluator_http_agglomerative.calculate_outliers_accuracy()

In [ ]:
all_labels